## Import libs
~ 10 min

In [43]:
%pip install -r requirements.txt
from IPython import display
display.clear_output()
print("all libs installed")

all libs installed


## Install Yolo, Tracker

In [45]:
%cd {HOME}
!git clone https://github.com/ultralytics/yolov5

%cd {HOME}/yolov5
%pip install -r requirements.txt

from IPython import display
display.clear_output()
print("yolo loaded")

In [46]:
%cd {HOME}
!git clone https://github.com/ifzhang/ByteTrack.git
# %cd {HOME}/ByteTrack

# # workaround related to https://github.com/roboflow/notebooks/issues/80
# !sed -i 's/onnx==1.8.1/onnx==1.14.0/g' requirements.txt
# !sed -i 's/onnxruntime==1.8.0/onnxruntime==1.12.0/g' requirements.txt

# %pip install -q -r requirements.txt
# !python3 setup.py -q develop
# %pip install -q cython_bbox
# %pip install -q onemetric
# # workaround related to https://github.com/roboflow/notebooks/issues/112 and https://github.com/roboflow/notebooks/issues/106
# %pip install -q loguru lap thop

# from IPython import display
# display.clear_output()
print("tracker loaded")

/home/jupyter/work/resources/crowd-head-counter


Cloning into 'ByteTrack'...
Checking out files: 100% (229/229), done.


In [44]:
#!g1.1
import torch
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

import os
HOME = os.getcwd()
print(HOME)

import pandas as pd
import time
import cv2
import supervision

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
torch:  2.0 ; cuda:  cu117
/home/jupyter/work/resources/crowd-head-counter


In [47]:
#!g1.1
import sys
sys.path.append(f"{HOME}/ByteTrack")

import yolox
print("yolox.__version__:", yolox.__version__)
%cd {HOME}

yolox.__version__: 0.1.0
/home/jupyter/work/resources/crowd-head-counter


## Config Tracker, Supervision

In [48]:
#!g1.1
from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass


@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 30
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False

In [49]:
#!g1.1
import supervision
from supervision.draw.color import ColorPalette
from supervision.geometry.core import Point
from supervision.utils.video import VideoInfo
from supervision.utils.video import get_video_frames_generator
from supervision.utils.video import process_video
from supervision.utils.notebook import plot_image as show_frame_in_notebook
from supervision.detection.annotate import Detections, BoxAnnotator
from supervision.detection.line_counter import LineZone as LineCounter, LineZoneAnnotator as LineCounterAnnotator

## Load & Config Model

In [50]:
#!g1.1
# !gdown  1gglIwqxaH2iTvy6lZlXuAcMpd_U0GCUb # load crowdhuman_yolov5m.pt
import gdown

gdrive_file_id = '1gglIwqxaH2iTvy6lZlXuAcMpd_U0GCUb'
dst_path = './crowdhuman_yolov5m.pt'

gdown.download(id=gdrive_file_id, output=dst_path)

Downloading...
From: https://drive.google.com/uc?id=1gglIwqxaH2iTvy6lZlXuAcMpd_U0GCUb
To: /home/jupyter/work/resources/crowd-head-counter/crowdhuman_yolov5m.pt
100%|██████████| 169M/169M [00:02<00:00, 63.8MB/s] 


'./crowdhuman_yolov5m.pt'

In [51]:
#!g1.1
model = torch.hub.load('ultralytics/yolov5', 'custom', path='./crowdhuman_yolov5m.pt', force_reload=True)

/home/jupyter/.local/lib/python3.8/site-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /tmp/xdg_cache/torch/hub/master.zip
YOLOv5 🚀 2023-7-5 Python-3.8.12 torch-2.0.1+cu117 CUDA:0 (Tesla V100-PCIE-32GB, 32510MiB)

Fusing layers... 
Model summary: 308 layers, 21041679 parameters, 0 gradients
Adding AutoShape... 


## Some utils for work

In [52]:
#!g1.1
def resize_video(in_path, out_path, height:int):
  import moviepy.editor as mp
  clip = mp.VideoFileClip(in_path)
  clip_resized = clip.resize(height=height) # According to moviePy documenation The width is then computed so that the width/height ratio is conserved.
  clip_resized.write_videofile(out_path)
  return


In [53]:
#!g1.1
def load_from_yadisk(file_key, dist_path):
    import requests
    from urllib.parse import urlencode

    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    public_key = file_key

    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']
    response = requests.get(download_url)

    with open(dist_path, 'wb') as f:
        f.write(response.content)
    print('file loaded')

In [54]:
#!g1.1
from typing import List

import numpy as np


# converts Detections into format that can be consumed by match_detections_with_tracks function
def detections2boxes(detections: Detections) -> np.ndarray:
    return np.hstack((
        detections.xyxy,
        detections.confidence[:, np.newaxis]
    ))


# converts List[STrack] into format that can be consumed by match_detections_with_tracks function
def tracks2boxes(tracks: List[STrack]) -> np.ndarray:
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=float)


# matches our bounding boxes with predictions
def match_detections_with_tracks(
    detections: Detections,
    tracks: List[STrack]
) -> Detections:
    if not np.any(detections.xyxy) or len(tracks) == 0:
        return np.empty((0,))

    tracks_boxes = tracks2boxes(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detections.xyxy)
    track2detection = np.argmax(iou, axis=1)

    tracker_ids = [None] * len(detections)

    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            tracker_ids[detection_index] = tracks[tracker_index].track_id

    return tracker_ids

In [55]:
#!g1.1
# returns real coord of line points from normalised coords
def make_line(video_path, p1_n, p2_n):
  V_PARAMS = VideoInfo.from_video_path(video_path)
  w, h = V_PARAMS.width, V_PARAMS.height
  return Point(int(w*p1_n[0]), int(h*p1_n[1])), Point(int(w*p2_n[0]), int(h*p2_n[1]))

In [56]:
#!g1.1
def process_frame(frame: np.ndarray, index: int) -> np.ndarray:
        start_time = time.time()
        results = model(frame)
        detections = Detections.from_yolov5(results)

        # filtering out detections with unwanted classes
        detections = detections[(detections.class_id == 1)]

        # tracking detections
        tracks = byte_tracker.update(
            output_results=detections2boxes(detections=detections),
            img_info=frame.shape,
            img_size=frame.shape
        )
        tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)
        detections.tracker_id = np.array(tracker_id)

        # filtering out detections without trackers
        detections = detections[(detections.tracker_id != None)]


        # updating line counter
        line_counter.trigger(detections=detections)
        line_counter.in_count

        c_in=line_counter.in_count
        c_out=line_counter.out_count

        # annotate frame
        if (SHOW_LABELS):
          # format custom labels
          labels = [
              f"#{tracker_id}"
              for _,_, confidence, class_id, tracker_id
              in detections
          ]
          frame = box_annotator.annotate(scene=frame, detections=detections, labels=labels)
        else:
          frame = box_annotator.annotate(scene=frame, detections=detections, skip_label=True)

        if (SHOW_LINECOUNTER):
          line_annotator.annotate(frame=frame, line_counter=line_counter)
        else:
          cv2.putText(frame, f"{c_in} {c_out}", (20, 20),	cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
          cv2.line(frame, (LINE_START.x,LINE_START.y) , (LINE_END.x, LINE_END.y), (255, 255, 255), 1)

        # Save to storage
        DS[index/FPS]=(c_in, c_out)

        # Show process stat
        print(f'frame {index}/{TOTAL_FRAMES} {len(detections)} {(time.time() - start_time):.4f}s {c_in=} {c_out=}')

        return frame

# Config all stuff here!!!


In [57]:
#!g1.1
point_start, point_end = [0.66, 0.42],[0.86, 0.44] # get normalized points it from https://roboflow.github.io/polygonzone/
video_link_from_yadisk = 'https://disk.yandex.ru/i/lV4PDmiVBBCklg'
out_path =  'out.mp4'
stat_path =  'out.csv'
LBL_SIZE: float = 0.5
LBL_I : int = 1

## Load and process video

In [58]:
#!g1.1
# if you want load video from gdrive do thomething like here
#%cd %cd {HOME}
#gdown.download(id='1kxBHYgVfgyRJsCaXUCotNt8WqUOuvWfU', output='./in.mp4')


# some demo videos
# !gdown 1h62b_Dh4Fw8ff2gwBLPC4WemT4MNE__x -O in.mp4 # subway_demo.mp4
# !gdown 1qZ6ROKdzHbQiHdizKfYbecr9qquOQ0Cz -O in.mp4 # subway.mp4 https://drive.google.com/file/d//view?usp=drive_link
# !gdown 1sGr3TgnFVsMrF6i0Eb2rkNxQHS7YdBxV -O in.mp4
# !gdown 1kxBHYgVfgyRJsCaXUCotNt8WqUOuvWfU -O in.mp4

In [63]:
#!g1.1
IN_VIDEO_PATH = f"{HOME}/in.mp4"

load_from_yadisk(video_link_from_yadisk, IN_VIDEO_PATH)


VIDEO_INFO = VideoInfo.from_video_path(IN_VIDEO_PATH)
FPS = VIDEO_INFO.fps
TOTAL_FRAMES = VIDEO_INFO.total_frames
SHOW_LABELS : bool = False
SHOW_LINECOUNTER: bool = False

# statistics storage
DS = {}

in_path = IN_VIDEO_PATH
print(VIDEO_INFO)

LINE_START, LINE_END = make_line(in_path, point_start, point_end)
print(LINE_START, LINE_END)

# create BYTETracker instance
byte_tracker = BYTETracker(BYTETrackerArgs())
# create VideoInfo instance
video_info = VideoInfo.from_video_path(in_path)
# create frame generator
generator = get_video_frames_generator(in_path)
# create LineCounter instance
line_counter = LineCounter(start=LINE_START, end=LINE_END)
# create instance of BoxAnnotator and LineCounterAnnotator
box_annotator = BoxAnnotator(color=ColorPalette.from_hex(['#00ff00']), thickness=LBL_I, text_thickness=LBL_I, text_scale=LBL_SIZE)
line_annotator = LineCounterAnnotator(thickness=LBL_I, text_thickness=LBL_I, text_scale=LBL_SIZE)

process_video(
    source_path=in_path,
    target_path=out_path,
    callback=process_frame
)
display.clear_output()
print(f"{VIDEO_INFO}, {LINE_START=}, {LINE_END=}, inout={DS[list(DS.keys())[-1]]}")

VideoInfo(width=596, height=336, fps=25, total_frames=362) Point(x=393, y=141) Point(x=512, y=147) (0, 15)


In [62]:
#!g1.1
# Write dictionary to CSV
with open(stat_path, 'w', newline='') as file:
    import csv
    writer = csv.writer(file)
    writer.writerow(['Time', 'in', 'out'])  # Optional: Write header
    for key, value in DS.items():
        writer.writerow([key, value[0], value[1]])